# Montpellier Open Street Map Data

Open Street Map is a map created using data contributed by volunteers around the world; local mappers and GIS professionals keep the maps up to date by uploading and editing information about different roads and landmarks around the world.

I've downloaded the .osm data for the map of the Montpellier area in France. I cleaned the data to fix invalid street, city, and postal code names and reformatted the data to store it in a tabular format **(Section 1)**. After downloading the data into .csv files, I uploaded them into a SQL database to investigate some different attributes of the data **(Section 2)**.

Querying the node tags allowed me to understand general features of the city, such as popular cuisine and tree species. **(Section 3)**.

## Section 1: Cleaning Montpellier Address Data

The .osm file for Montpellier is 199,267 KB so I chose to use cElementTree with the iterparse method to work with the .osm data. I created a smaller sample file to use when validating code functionality (SAMPLE_FILE).

Below, I've created a dictionary containing all tags in the XML dataset, and the number of each type. These tags refer to elements within the Open Street Map (OSM) data, as well as tags for each element. 

OSM XML is made up of three different elements: '

1. Nodes - defined location made up of an ID and pair of coordinates
2. Ways - paths between nodes, ways to get places, linear features, boundaries
3. Relations - explains how elements work together, often an ordered list of nodes/ways

The function 'count_tags' counts the number of different elements and tags (used to describe features of elements) in the file.   

Output: {'bounds': 1,
 'member': 19106,
 'nd': 1196463,
 'node': 854439,
 'osm': 1,
 'relation': 1610,
 'tag': 460554,
 'way': 140286}

Each tag has a key describing the tag attribute, which is held as the 'value'. The function key_type is used to understand how many of the keys have problem characters, or colons (which may be used to nest information).

```
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
lower_double_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
```
Output: {'lower': 433135,
 'lower_colon': 25148,
 'lower_double_colon': 335,
 'other': 1934,
 'problemchars': 2}

## Cleaning Street Names

The first cleaning task involved making the street names consistant.

The list of 'expected' street names shows the words that I expected to find that describe different streets in Montpellier. By looking through a map of the city, I came up with some expected words such as 'rue' (translates to 'road'), 'avenue', 'boulevard', 'route', 'chemin' (translates to 'path'). In the French language, these words tend to come at the beginning of the road title (e.g. Rue Ferdinand Fabre).

For roads that may have been abbreviated, I created the dictionary 'mapping' to update abbreviations to the full name.
```
mapping = { "av.": "Avenue",
            "ave": "Avenue",
            "Ave.":"Avenue",
            "R.":"Rue",
            "r.":"Rue",
            "blvd":"Boulevard",
            "blvd.":"Boulevard"
            }
```
The map and list of expected words was used to update street names using the 'update_name' function as data was being put into the SQL database.
```
def update_name(name, mapping):
    updated_name_list=[]
    name_list=name.split()
    updated_names=name
    
``` 
I first checked to see if the road name begins in middle of phrase (e.g. 11, Rue Ferdinand Fabre). For phrases that include the building name or number before the street name, every part of the address before the street name is removed.
```
    for road in expected:
        if road in name_list:
            i=0
            for name in name_list:
                if name==road:
                    updated_name_list=name_list[i:]
                    updated_names=' '.join(updated_name_list)
                    break
                else:
                    i+=1```
                    
                    
            
If this was not the case, I checked if the first word can be mapped to a different word in the correction list for mapping.

```

    if name_list[0] in mapping.keys():
            name_list[0] = mapping[name_list[0]]
            updated_names=' '.join(name_list)
    return updated_names
    
```


One thing that became clear when looking through the sources of this data, was that the french accents were incoded as a special value. The UCF-8 characters are recorded as ASCII strings. I decided to leave the encoded text, but have accounted for it throughout the cleaning procedure. For example, the word u"All\xe9e", the UCF-8 encoding for allée, was added to the list of expected roads (translating to alley in French).

#### Using the same method as above, I investigated the city, country, and postal code of each of these tags.

All country codes were properly labelled "FR" for France.

For the postal codes, I expected to see the four codes attributed to the city of Montpellier: "34000","34070","34080","34090"

Besides these codes, I list of postal codes for surrounding communes was returned ['34130', '34170', '34006', '34920', '34790', '34970', '34880', '34990', '34830']. This makes sense because in pulling the OSM data for Montpellier, I selected a rectangular region. This would have included communes outside of the city of Montpellier.

#### For this reason, I decided to assess the validity of the data by filtering the postal codes only by the département of Hérault and postal code length (5 characters). 
In French postal codes, the first two numbers refer to the département (34 in this case). All postal codes in this dataset began with the digits '34', but some were longer than 5 characters (e.g. '34064 Montpellier Cedex 2'). For all postal codes greater than 5 characters, I used a function to return only the first 5 characters of the postal code.

```def investigate_zip(zip_value,code_list):
    new_code=zip_value
    if len(zip_value)>5: #check for postcodes with text appended 
        code_list.add(tag.attrib['v']) #add to list for reference
        new_code=new_code[:5]
    if zip_value[:2] !='34': #check for non-Herault codes
        new_code='error' #entry to be discluded later
        code_list.add(tag.attrib['v'])
    return new_code
    ```
    


In order to fix city data, I looked at the different cities in the dataset, and added valid communes to the list of valid cities. For common typos, or areas within a hamlet, I setup a dictionary that could be used to fix the names or assign the appropriate hamlet.
```
#create dictionary of things to change faulty street names to
    mapping_city = { "Castelnau le Lez": "Castelnau-le-Lez",
                "Montpelier": "Montpellier",
                "Saint-Jean-de-Vedas":u'Saint-Jean-de-V\xe9das',
                "Montpelle":"Montpellier",
                'Castelnau le lez':"Castelnau-le-Lez",
                'Castelnau le Les':"Castelnau-le-Lez",
                 u'Saint Cl\xe9ment de riviere':u'Saint-Cl\xe9ment-de-Rivi\xe8re',
                'Maurin':'Lattes'
                }```

## Section 2. Parsing data into SQL Database

In order to put this data into an SQL database, I will parse each element in the XML file, putting them into a tabular format that can be written to a .csv file. I'm using a schema and validation library to check the data before writing the data structures to new .csv files.

The "node" field has a dictionary with the following attributes:
- id, user ,uid, version, lat, lon, timestamp, changeset

The "node_tags" contains a list of dictionaries of different tags for each node containing the following attributes:
- id: id of the top level node
- key: characters before a colon are removed and added to the 'type' field
- value
- type:  "regular" if a colon is not present.

The dictionaries for the 'way' field contain the following attributes. There is also a list of tags formatted in the same form as for the node tags.

- id, user, uid, version, timestamp, changeset

The "way_nodes" holds list of dictionaries, one for each nd child tag.  Each dictionary has the following attributes:
- id
- node_id
- position: index to reference position

```         
            if PROBLEMCHARS.search(minitag.attrib["k"])==None:
                
                #first,fix streetname
                if minitag.attrib['k'] == "addr:street":
                    #change road name to fixed name 
                    minitag.attrib['v']=audit_street_type(street_types, minitag.attrib['v'])```

Keys and attributes are then added to the dictionary and split if a colon was present in the key.

```           
      
                dic["id"]=element.attrib["id"]
                if ":" in minitag.attrib["k"]:
                    splitkey=minitag.attrib["k"]
                    splitkey=splitkey.split(":",1)
                    dic["key"]=splitkey[1]
                    dic["type"]=splitkey[0]
                else:
                    dic["key"]=minitag.attrib["k"]
                    dic["type"]="regular"
                dic['value']=minitag.attrib['v']
            ```
        

Once the .csv files were written, I imported them into a sql database using sqlite3. I chose to use python to specify the data types and include the program in the same workflow. The processed used to create the database files and tables is shown below. It was repeated for each of the five tables.


Final database files:

nodes.db : 64,273 KB
nodes_tags.db : 4,340 KB
ways.db : 16,087 KB
ways_tags.db : 45,493 KB
ways_nodes.db : 22,903 KB

montpellier.db: 122,303 KB

During this process, I also counted the number of unique nodes and ways in the dataset. There's a total of 854,439 nodes (see count query below) and 140,286 ways. It makes sense that there are more nodes, as each way is made up of a collection of nodes.


## Section 3. User Contribution Analysis

Below is the code used to count how many users contributed to the Montpellier OSM data. There was a total of 714 entries that had distinct user IDs.

```#select user entry data
c.execute('SELECT user,COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e GROUP BY e.user ORDER BY num DESC;')
all_rows = c.fetchall()```

After combining user ID list with the respective number of entries, I looked at some summary statistics for the DataFrame.

Attribute|Value
---|---
mean  |     1393.172269
std    |   16322.682447
min     |      1.000000
25%      |     2.000000
50%       |    7.000000
75%        |  36.000000
max     | 392261.000000

The max user contributed 392261 unique nodes/ways, but 75% of users contributed 36 or fewer OSM entries. This leads me to believe that a subset of users are entering data programatically, while the majority of users enter this data manually.

## Landmarks in Montpellier

I began by running a SQL query to understand the different tags used to describe different nodes, pulling the 300 most popular entries to browse. I looked at the most popular values for tags that interested me.

### Food

```
#investigate tag types
c.execute('SELECT key, COUNT(*) FROM node_tags GROUP BY key ORDER BY COUNT(*) DESC LIMIT 10;')
```

It was interesting to compare the french and american cultures. For example, although kebabs are the 3rd most common cuisine in Montpellier. Traditional kebab restaurants are rarely eaten in the United States. Their popularity stems from the introduction of the kebab Turkish immigrants and popularity with the North African French population (http://www.reuters.com/article/us-france-immigration-kebabs-idUSKBN0IH0CQ20141028).

```
c.execute('SELECT value, COUNT(*) as num FROM node_tags WHERE key="cuisine" GROUP BY value ORDER BY num DESC LIMIT 10;')
all_rows = c.fetchall()
pprint(all_rows)
print("\n")
```
### Shops

Bakeries and butchers are among the most common stores in Montpellier. Unlike in many cities in the United States, French locals often choose to buy their bread/pastries and meat at stores separate from a large supermarket.

```
c.execute('SELECT value, COUNT(*) as num FROM node_tags WHERE key="shop" GROUP BY value ORDER BY num DESC LIMIT 10;')
```

Output: [(u'clothes', 191),
 (u'hairdresser', 183),
 (u'bakery', 164),
 (u'convenience', 108),
 (u'butcher', 87),
 (u'estate_agent', 86),
 (u'furniture', 85),
 (u'beauty', 67),
 (u'newsagent', 63),
 (u'greengrocer', 59)]

### Tree Species

I also looked at the most common tree species in Montpellier, learning that 'Platanus x acerifolia', or the London Planetree, is the most common species in the city. These types of trees are popular within cities due to their resistance to warm weather and pollution.

```
c.execute('SELECT value, COUNT(*) as num FROM node_tags WHERE key="species" GROUP BY value ORDER BY num DESC LIMIT 5;')
```
Output: [(u'Platanus x acerifolia', 113),
 (u'pinus pinea', 75),
 (u'Pin Pignon', 70),
 (u'Micocoulier de Provence', 32),
 (u'C. sempervirens', 16)]

## Taking a Digital Tour of the City

I was interested in understanding what types of trees were in the city and I enjoy learning about the species the grow in different regions of the world. This data could certainly be used to find different species of trees within the city.

The tag ID can be used to join with the parent node ID and recover the latitude and longitude of any specific tree. This could be a useful tool for tree enthusiasts who want to know more about what areas of the city the Mediterannean Cyprus grows, and where they could see one in person.

Since many OSM contributers find their data using aerial photography and GPS, it's possible that these species were extracted using imprecise methods using only size or color of the tree. Information could be validated and would need to be updated semi-regularly as new trees are planted.

A platform could be created for users to walk around the city, being able to see landmarks such as trees or finding different types of cuisne. The GPS data is readily available and can be extracted to find the appropriate GPS coordinates for different landmarks in a city.

### Other Ways to Improve Data Quality

One way to improve the validity of the data, as in the case of correcting incorrect postal codes, would be to use an application to 'inspect' certain tag attributes after they've been created. For users generating large amounts of data, a function such as the function 'investigate_zip' could be used to recognize postal codes that do not meet a decided postal code attribute specification.

Data could also be inspected for uniqueness. If two nodes have latitudes and longitudes similar to a certain degree, the nodes could be recognized with an inspection function to allow the user to confirm whether the nodes are the same (and one can be removed), different (and the location could be changed), or different objects in very close proximity.
